In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import base64
import os
import time
from dotenv import load_dotenv
import requests
import json
import csv

### 환경 변수 로드

In [2]:
load_dotenv()

True

### Heyhome API 설정

##### Address : https://goqual.notion.site/API-078d8c3c22f24b23bda799e38f2c819d

In [3]:
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
APP_KEY = os.getenv('APP_KEY')
USERNAME = os.getenv('USERNAME')
PASSWORD = os.getenv('PASSWORD')
REDIRECTURI = os.getenv('REDIRECTURI')
BASE_URL = os.getenv('BASE_URL')

In [4]:
# print(CLIENT_ID, CLIENT_SECRET, APP_KEY, USERNAME, PASSWORD, REDIRECTURI, BASE_URL)

## AES256 암호화 클래스

In [5]:
class AES256:
    def __init__(self, appKey):
        self.appKey = appKey

    def encrypt(self, text):
        key = self.appKey[:32].encode('utf-8')
        iv = self.appKey[:16].encode('utf-8')
        cipher = AES.new(key, AES.MODE_CBC, iv)
        encrypted = cipher.encrypt(pad(text.encode('utf-8'), AES.block_size))
        return base64.urlsafe_b64encode(encrypted).decode('utf-8')

    def decrypt(self, cipherText):
        key = self.appKey[:32].encode('utf-8')
        iv = self.appKey[:16].encode('utf-8')
        cipher = AES.new(key, AES.MODE_CBC, iv)
        decodedBytes = base64.urlsafe_b64decode(cipherText)
        decrypted = unpad(cipher.decrypt(decodedBytes), AES.block_size)
        return decrypted.decode('utf-8')

### Heyhome API 연결 설정

In [6]:
aes256 = AES256(APP_KEY)

In [7]:
json_data = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "grant_type": "password",
    "username": USERNAME,
    "password": PASSWORD
}

### 데이터 암호화

In [8]:
encrypted_data = aes256.encrypt(json.dumps(json_data))
encrypted_data

'AA6Twh14cgrcahuLYjU3f0ReyDF6aUsGsT5jDTkg8AjTeJJOv_z8W-SGObovlw_OFkd1JGTX0g6BRTbvB4bXcVyDDdFt2OX_iXoO6t953HEsyKW9_v77yYqWhnmlH8YuInTzuwafaZ7zoRIWvSOb7yoWaoNeJX3aqEkSlmYc8xYN0gxVoqej_qlztGj_8MF_KtO2bkd311QDiaXfNeLE96ID3RLtFpPB5BvYeaOFUwU_SbFidUG0fv2IKtCL-Ymh'

### Heyhome API 토큰 발급

In [9]:
token_url = f"{BASE_URL}/token"
response = requests.post(
        token_url,
        json={"data": encrypted_data}
    )
response

<Response [200]>

In [ ]:
if response.status_code == 200:
    token_data = response.json()
# token_data

{'access_token': '81d9ebf7-dd6c-4918-bd5d-b0a26fd49cb7',
 'token_type': 'bearer',
 'refresh_token': '2414c30e-b918-4980-a6f3-daa4a4173694',
 'expires_in': 14324121,
 'scope': 'openapi'}

In [ ]:
ACCESS_TOKEN = token_data.get('access_token')
# ACCESS_TOKEN

'81d9ebf7-dd6c-4918-bd5d-b0a26fd49cb7'

### Heyhome Device List

#### https://goqual.notion.site/device-API-eee52e71bf1c4903b090b1330876c09d

In [12]:
def get_device_list(ACCESS_TOKEN, BASE_URL):
    """Fetch the list of devices from HeyHome API."""
    device_url = f"{BASE_URL}/devices"
    headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

    # API 호출
    response = requests.get(device_url, headers=headers)
    
    if response.status_code == 200:
        devices = response.json()
        # print("Device List:")
        # for device in devices:
        #     print(f"- ID: {device['id']}, Name: {device['name']}, Type: {device['deviceType']}")
        return devices
    else:
        print("Error fetching devices:", response.status_code, response.text)
        return None

In [ ]:
response = get_device_list(ACCESS_TOKEN, BASE_URL)
# response

[{'id': '47260522a4e57c9dd5cf',
  'name': '안방조명',
  'deviceType': 'LightRgbw5',
  'hasSubDevices': False,
  'modelName': 'GKW-MD081',
  'familyId': '49087331',
  'category': 'dj',
  'online': True},
 {'id': 'eb4ba5a3b2d1d893d8tkid',
  'name': '안개발생기',
  'deviceType': 'SmartPlugMini',
  'hasSubDevices': False,
  'modelName': '',
  'familyId': '49087331',
  'category': 'cz',
  'online': True},
 {'id': 'ebde650d0bafd44afcyggh',
  'name': '플라즈마',
  'deviceType': 'SmartPlugMini',
  'hasSubDevices': False,
  'modelName': '',
  'familyId': '49087331',
  'category': 'cz',
  'online': True},
 {'id': '50450710e8db84f198f8',
  'name': '멀티탭',
  'deviceType': 'PowerStrip2',
  'hasSubDevices': False,
  'modelName': '',
  'familyId': '49087331',
  'category': 'cz',
  'online': True},
 {'id': '80001764f4cfa2fdb3a1',
  'name': '책상',
  'deviceType': 'LightRgbw5',
  'hasSubDevices': False,
  'modelName': 'GKW-MD081',
  'familyId': '49087331',
  'category': 'dj',
  'online': True}]

### Heyhome Device 상태 조회 ('PowerStrip2', 'id': '50450710e8db84f198f8')

##### https://goqual.notion.site/Description-of-requirments-a7ad510d30b742fd8021e45e8f492546

```
GET /openapi/device/${deviceId}
Host: goqual.io
Authorization: Bearer ${access_token}
```

In [ ]:
device_id = response[3].get('id')
# device_id

'50450710e8db84f198f8'

In [15]:
# 저장할 토큰
data = {
    'token': ACCESS_TOKEN,
    'device_id': device_id
}

# JSON 파일로 저장
with open('token_deviceId.json', mode='w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

In [16]:
def get_device_status(ACCESS_TOKEN, BASE_URL, device_id):
    """Fetch the current status of a device."""
    status_url = f"{BASE_URL}/device/{device_id}"
    HEADERS = {
        "Authorization": f"Bearer {ACCESS_TOKEN}",
        "Content-Type": "application/json"
    }
    print(f"Fetching status for device {device_id}...")
    response = requests.get(status_url, headers=HEADERS)

    if response.status_code == 200:
        status = response.json()
        # print(f"Current status of device {device_id}:")
        # print(json.dumps(status, indent=4))
        return status
    else:
        print(f"Failed to fetch status for device {device_id}:", response.status_code, response.text)
        return None

In [ ]:
get_device_status(ACCESS_TOKEN, BASE_URL, device_id)

Fetching status for device 50450710e8db84f198f8...


{'id': '50450710e8db84f198f8',
 'deviceType': 'PowerStrip2',
 'deviceState': {'power1': False,
  'power2': False,
  'power3': False,
  'power4': True,
  'power5': True}}

### Heyhome Device 제어 ('PowerStrip2', 'id': '50450710e8db84f198f8')

```
POST /openapi/control/${deviceId}
Host: goqual.io
Authorization: Bearer ${access_token}
```

In [18]:
def control_power_strip_ports(ACCESS_TOKEN, BASE_URL, device_id, states):
    """
    멀티탭의 여러 포트를 제어하는 함수
    :param device_id: 제어할 장치의 ID
    :param states: 포트 상태를 나타내는 딕셔너리 (예: {"power1": True, "power2": False})
    """
    url = f"{BASE_URL}/control/{device_id}"
    HEADERS = {
        "Authorization": f"Bearer {ACCESS_TOKEN}",
        "Content-Type": "application/json"
    }
    payload = {
        "requirments": states  # 포트 상태를 전달
    }
    try:
        response = requests.post(url, headers=HEADERS, data=json.dumps(payload))
        if response.status_code == 200:
            print(f"Device {device_id} ports updated successfully.")
            return response
        else:
            print(f"Failed to update device ports. Status code: {response.status_code}, Response: {response.text}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [19]:
states = {
    'power1': False,
    'power2': False,
    'power3': False,
    'power4': True,
    # 'power5': True
}

In [20]:
control_power_strip_ports(ACCESS_TOKEN, BASE_URL, device_id, states)

Device 50450710e8db84f198f8 ports updated successfully.


<Response [200]>